In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests
from collections import defaultdict

logic:
pre-requirement: check whether more than two blocks left in current block
1. in current block, get the total-delegation, and total undelegation amount
2. in the last block in current epoch, get the new total-delegation and new total undelegation amount
3. get the total-delegation changes, and the difference of total undelegation amount changes
4. check whether the two changes are the same

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [5]:
def getEpoch():
    method = 'hmy_getEpoch'
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [6]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result'] 

In [7]:
def getStakeAndUndelegate(epoch):
    validator = dict()
    undelegate = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        address = i['validator']['address']
        validator[address] = i['total-delegation']
        undel = 0
        for d in i['validator']['delegations']:
            for j in d['undelegations']:
                if epoch == j['epoch']:
                    undel += j['amount']
        undelegate[address] = undel
    return validator, undelegate

In [8]:
def diffAndFilter(map1, map2):
    map3 = dict()
    for k, v in map2.items():
        if k in map1:
            if v - map1[k] != 0:
                map3[k] = v - map1[k]
    return map3

In [9]:
def U2_test():
    print("Test-U2: After undelegate, the total stake amount for that validator should subtract the undelegation amount before next epoch")
    num = int(input("enter the number of epoches you want to test: "))
    iterations = 0
    while iterations < num:
        
        block = getBlockNumber()
        last_block = getLastBlockOfCurrentEpoch()
        
        # need at least 2 blocks left to compare difference
        if block == last_block:
            new_block = last_block + 1
            while block < new_block:
                block = getBlockNumber()
        epoch = getEpoch()
        print("current epoch numebr: ", epoch, "block number: ", block, "will begin testing...")
        stake, undelegate = getStakeAndUndelegate(epoch)

        last_block = getLastBlockOfCurrentEpoch()
        print("last block in current epoch: ", last_block)
        while block < last_block:
            block = getBlockNumber()
        print("last block number reaches", block, "will compare the stakes and undelegations")
        new_stake, new_undelegate = getStakeAndUndelegate(epoch)
        diff_stake = diffAndFilter(stake, new_stake)
        diff_undelegate = diffAndFilter(undelegate, new_undelegate)

        if not diff_undelegate:
            print("no undelegation happens in current test, need more tests")
            
        for k,v in diff_undelegate.items():
            if k in diff_stake:
                if v == -(diff_stake[k]):
                    print("Test-U2: Success")
                else:
                    print("Validator ", k, "the stake change doesn't meet the undelegation change")
            else:
                print("Test-U2: Fail")
                print("Validator: ", k, "total stakes doesn't change after undelegation")
      
        iterations += 1        
    

In [10]:
U2_test()

Test-U2: After undelegate, the total stake amount for that validator should subtract the undelegation amount before next epoch
enter the number of epoches you want to test: 1
current epoch numebr:  61 block number:  2348 will begin testing...
last block in current epoch:  2355
last block number reaches 2355 will compare the stakes and undelegations
Test-U2: Success
